# Additional - Markov Chain with Image

Here is a small code snippet for running markov chain with image. Just play around with it.

Based on https://github.com/JonnoFTW/markov-img-gen

with the data from Bauhaus painting</br>
[<img src="images/paint.jpg" width="300x"/>](originalPic.jpeg)

[<img src="images/Markov_directional.jpeg" width="250x"/>](Markov_directional.jpeg)
[<img src="images/markovProcess_DFS.gif" width="250x"/>](markovProcess_DFS.gif)
[<img src="images/markovProcess_BFS.gif" width="250x"/>](markovProcess_BFS.gif)


In [1]:
!pip install pyprind

You should consider upgrading via the '/Users/AprilCoffee/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
from collections import defaultdict, Counter
from PIL import Image
import numpy as np
#import pyprind

from IPython.display import clear_output

import random
class MarkovChain(object):
    def __init__(self,bucket_size = 10, directional=False,method='DFS'):
        self.bucket_size = bucket_size
        self.weights = defaultdict(Counter)
        self.directional = directional
        if(method=='DFS' or method =='BFS'):
            self.method = method
        #DFS as default method
        else:
            self.method = 'DFS'
            
    def get_neighbors(self, x, y):
        return [(x, y - 1),(x, y + 1),(x - 1, y),(x + 1, y)]
    def get_neighbors_directional(self, x, y):
        return {'t':(x, y - 1), 'd':(x, y + 1), 'l':(x - 1, y), 'r':(x + 1, y)}
    
    def norm(self,_pixel):
        return _pixel // self.bucket_size
    def denorm(self,_pixel):
        return _pixel * self.bucket_size
    
    
    
    def train(self,img):
        width,height = img.size
        img = np.array(img)[:,:,:3]
        trainingCount = 0
        #prog = pyprind.ProgBar((width * height), width=64, stream=1)
        for x in range(height):
            for y in range(width):
                trainingCount += 1
                if(trainingCount%10000==0):
                    clear_output(wait=True)
                if(trainingCount%500==0):
                    print("{:.2%}".format(trainingCount/(width * height)))
                #prog.update()
                pixel = tuple(self.norm(img[x,y]))
                for neighbor in self.get_neighbors(x,y):
                    try:
                        self.weights[pixel][tuple(self.norm(img[neighbor]))]+=1
                    except IndexError:
                        continue
        self.directional = False
        
    def train_with_direction(self, img):
        self.weights = defaultdict(lambda: defaultdict(Counter))
        width, height = img.size
        trainingCount = 0
        img = np.array(img)[:, :, :3]
        #prog = pyprind.ProgBar((width * height), width=64, stream=1)
        for x in range(height):
            for y in range(width):
                trainingCount += 1
                if(trainingCount%10000==0):
                    clear_output(wait=True)
                if(trainingCount%500==0):
                    print("{:.2%}".format(trainingCount/(width * height)))
                #prog.update()
                pixel = tuple(self.norm(img[x, y]))
                for _dir, neighbor in self.get_neighbors_directional(x, y).items():
                    try:
                        self.weights[pixel][_dir][tuple(self.norm(img[neighbor]))] += 1
                    except IndexError:
                        continue
        self.directional = True
        
    def generate(self,initial=None,width=400, height=400):
        
        if initial is None:
            initial = random.choice(list(self.weights.keys()))
        #print(initial)
            
        img = Image.new('RGB',(width,height),(255,255,255))
        img = np.array(img)
        img_out = np.array(img.copy())
        
        start_position = (random.randint(0,height),random.randint(0,width))
        img[start_position] = initial
        stack = [start_position]
        isFilled = set()
        
        #prog = pyprind.ProgBar((width * height), width=64, stream=1)
        trainingCount = 0
        
        i = 0
        
        while stack:
            
            if(self.method=='DFS'):
                x,y = stack.pop()
            elif(self.method=='BFS'):
                x,y = stack.pop(0)
            else:
                x,y = stack.pop()
                
            if(x,y) in isFilled:
                continue
            else:
                isFilled.add((x,y))
                
            trainingCount += 1
            if(trainingCount%10000==0):
                clear_output(wait=True)
            if(trainingCount%500==0):
                print("{:.2%}".format(trainingCount/(width * height)))                
            #prog.update()
            pixel = img[x,y]
            node = self.weights[tuple(pixel)]
            img_out[x,y] = self.denorm(pixel)
            
            #uncomment to generate sequence image
            #Image.fromarray(img_out).save('frames/'+"{:05d}".format(i)+'.jpg',dpi=[72,72])
            i+=1
            
            if self.directional:
                keys = {dir: list(node[dir].keys()) for dir in node}
                neighbors = list(self.get_neighbors_directional(x, y).items())
                counts = {dir: np.array(list(node[dir].values()), dtype=np.float32) for dir in keys}
                key_index = {dir: np.arange(len(node[dir])) for dir in keys}
                probabilities = {dir: counts[dir] / counts[dir].sum() for dir in keys}
            
            #BFS without Direction
            else:
                keys = list(node.keys())
                neighbors = self.get_neighbors(x,y)
                counts = np.array(list(node.values()),dtype=np.float32)
                key_index = np.arange(len(keys))
                probabilities = counts / counts.sum()

            # shuffle all directions
            np.random.shuffle(neighbors)
            
            for neighbor in neighbors:
                try:
                    if self.directional:
                        direction = neighbor[0]
                        neighbor = neighbor[1]
                        if neighbor not in isFilled:
                            col_idx = np.random.choice(key_index[direction], p=probabilities[direction])
                            img[neighbor] = keys[direction][col_idx]
                    else:
                        col_idx = np.random.choice(key_index, p=probabilities)
                        if neighbor not in isFilled:
                            img[neighbor] = keys[col_idx]
                            
                except IndexError:
                    pass
                except ValueError:
                    continue
                    
                if 0 <= neighbor[1] < width and 0 <= neighbor[0] < height:
                    stack.append(neighbor)
                    
        return Image.fromarray(img_out)


## adding the training picture

In [ ]:
image = Image.open("images/paint.jpg")

#bucket the pixel for compressing image RGB
bucket_size = 10

bucketed = Image.fromarray((np.array(image)//bucket_size) * bucket_size)
#bucketed.show()

## Training
### Arguments
`bucket_size`   | how much the image is compressed (0~255) </br>
`directional`   | whether the program consider relations between different directions </br>
`method`        | `BFS` and `DFS`, different algorithm for traversing graph while generating. </br>

In [ ]:
chain = MarkovChain(bucket_size=bucket_size, 
                    directional=False,
                    method = 'DFS')


# pick one method to train
chain.train_with_direction(bucketed) 
#chain.train(bucketed)

19.48%
19.50%
19.52%
19.54%
19.56%
19.58%
19.60%
19.62%
19.64%
19.66%


## Show the image

In [4]:
output = chain.generate(width=400, height=400)
output.show()

60.00%
60.06%
60.12%
60.19%
60.25%
60.31%
60.38%
60.44%


KeyboardInterrupt: 